In [ ]:
from fetching.missy_fetching import get_vertretungen
import os
from dotenv import load_dotenv
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from config import update_cache
from fetching.missy_fetching import get_distances, get_clients, get_mas, get_prio_assignments
from fetching.experience_logging import get_experience_log

from data_processing.data_processor import DataProcessor
from optimize.optimize import Optimizer
from learning.model import AbnormalityModel
from data_processing.features_retrieval.create_single_df import create_single_df
from data_processing.features_retrieval.create_replacements import create_replacements
load_dotenv(override=True)

user = os.getenv("USER")
pw = os.getenv("PASSWORD")

In [24]:
from analysis import create_time_series_plots, create_comparison_plots

In [2]:

# Retrieve mostly static data
distances = get_distances(user, pw, update_cache=update_cache)
clients = get_clients(user, pw, update_cache=update_cache)
mas = get_mas(user, pw, update_cache=update_cache)
prio_assignments = get_prio_assignments(user, pw, update_cache=update_cache)
experience_log = get_experience_log()

In [ ]:
data_processor = DataProcessor(
    mas, clients, prio_assignments, distances, experience_log
)
comparison = []
start_date = "2025-03-01"
end_date = "2025-03-31"
for relevant_date in pd.date_range(start=start_date, end=end_date):
    relevant_date = relevant_date.strftime("%Y-%m-%d")

    vertretungen = get_vertretungen(
        relevant_date, user, pw, use_cache=False, update_cache=False
    )

    if len(vertretungen) == 0:
        continue

    relevant_date = datetime.strptime(relevant_date, "%Y-%m-%d")

    free_ma_records = list(
        filter(lambda x: x.get("mavertretend") != None, vertretungen)
    )
    open_client_records = list(
        filter(lambda x: x.get("klientzubegleiten") != None, vertretungen)
    )

    free_ma_ids = [elem["mavertretend"]["id"] for elem in free_ma_records]
    open_client_ids = [
        elem["klientzubegleiten"]["id"] for elem in open_client_records
    ]
    assignments = [
        {
            "ma": elem["mavertretend"]["id"],
            "klient": elem["klientzubegleiten"]["id"],
        }
        for elem in vertretungen
        if elem.get("mavertretend") != None
        and elem.get("klientzubegleiten") != None
    ]

    clients_df, mas_df = data_processor.create_day_dataset(
        open_client_ids, free_ma_ids, relevant_date
    )

    abnormality_model = AbnormalityModel()

    optimizer = Optimizer(mas_df, clients_df, abnormality_model)
    optimizer.create_model()

    optimizer.solve_model()
    assigned_pairs, recommendation_id = optimizer.process_results()

    assigned_pairs = {elem["ma"]: elem["klient"] for elem in assigned_pairs}
    assignments = {elem["ma"]: elem["klient"] for elem in assignments}

    replacements = create_replacements(assignments)
    replacement_recommendations = create_replacements(assigned_pairs)

    single_df_labels = create_single_df(
        clients_df, mas_df, replacements, relevant_date
    )
    single_df_recommendations = create_single_df(
        clients_df, mas_df, replacement_recommendations, relevant_date
    )

    # print(f"Labels: {single_df_labels.describe()}")
    # print(f"Recommendations: {single_df_recommendations.describe()}")

    # Create a dictionary that retrieves entries from the describe function
    description_labels = single_df_labels.describe().to_dict()
    description_recommendations = single_df_recommendations.describe().to_dict()

    my_dict = {
        "date": relevant_date,
        "count_labels": len(single_df_labels),
        "count_recommendations": len(single_df_recommendations),
    }

    # Loop through all keys in the description dictionaries
    for key in description_labels.keys():
        if key == "date":
            continue
        my_dict[f"{key}_labels"] = {
            "mean": description_labels[key]["mean"],
            "median": description_labels[key]["50%"],
            "std": description_labels[key]["std"],
            "min": description_labels[key]["min"],
            "max": description_labels[key]["max"],
        }
        my_dict[f"{key}_recommendations"] = {
            "mean": description_recommendations[key]["mean"],
            "median": description_recommendations[key]["50%"],
            "std": description_recommendations[key]["std"],
            "min": description_recommendations[key]["min"],
            "max": description_recommendations[key]["max"],
        }

    comparison.append(my_dict)


In [ ]:
# print(my_dict)
# create_comparison_plots(my_dict)
create_time_series_plots(comparison)